# Homework 2: Linear Algebra

### *Ifigeneia Giannakoudi*


##  <span style="color:brown"> 1. As demonstrated in the lecture, approximate fstar(x) = sin(x) on the interval [0, pi/2] via polynomials using a least squares approach. Use 1,000 evaluation points. </span></p>

In [141]:
import Pkg; Pkg.add("Plots")

   Resolving package versions...
   Installed GR_jll ─────────────────────── v0.71.7+0
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Xorg_xcb_util_jll ──────────── v0.4.0+1
   Installed Xorg_xkbcomp_jll ───────────── v1.4.2+4
   Installed Measures ───────────────────── v0.3.2
   Installed Xorg_libxkbfile_jll ────────── v1.1.0+4
   Installed RecipesPipeline ────────────── v0.6.11
   Installed Xorg_xcb_util_keysyms_jll ──── v0.4.0+1
   Installed Xorg_libXinerama_jll ───────── v1.1.4+4
   Installed Qt5Base_jll ────────────────── v5.15.3+2
   Installed xkbcommon_jll ──────────────── v1.4.1+0
   Installed Pipe ───────────────────────── v1.3.0
   Installed PlotThemes ─────────────────── v3.1.0
   Installed fzf_jll ────────────────────── v0.29.0+0
   Installed GLFW_jll ───────────────────── v3.3.8+0
   Installed Xorg_libXcursor_jll ────────── v1.2.0+4
   Installed JLFzf ──────────────────────── v0.1.5
   Installed Xorg_l

In [143]:
using WGLMakie

In [2]:
import Pkg; Pkg.add("LaTeXStrings")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [b964fa9f] + LaTeXStrings v1.3.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using LaTeXStrings

Our goal is to approximate a function, in this case $f^*(x)=sinx$ with polynomials in the range $x=[0,\pi/2]$, so we want to write it as an expansion series:

$ g(x) = \sum_{n=0}^{n_{max}} c_n x^n$

where $n_{max}$ is the order of the polynomial we use for the approximation and $c_n$ are the coefficients of the expansion. We are looking for the values of these coefficients that will lead to the best fitting. We divide our range $[0,\pi/2]$ to npoints.To find them we will use the method of Least Squares, i.e. we want to minimize summation on all the npoints of the square of the difference between the value $f^*(x_i)$ and $g(x_i)$.
The expression we want to minimize:
$G = \sum_{i=0}^{N-1}( f(x_i) - \sum_{n=0}^{n_{max}} c_n x_i^n )^2$

We can write this expression as a Linear Algebra equation, using matrices and vectors as:

$
    f(x_i) - \sum_{n=0}^{n_{max}} c_n x_i^n \rightarrow |b\rangle - A |c\rangle
$

where $|b\rangle$ is the vector whose elements are $f(x_i)$, $|c\rangle$ is the vector whose elements are the coefficients $c_n$ and $A$ is a matrix whose element [i,j] is $(x_{i-1})^{j-1}$, the (i-1)th point in the (j-1)-th power. I write i-1 and j-1, because Julia starts counting the elements from 1, but our expansion series begins from n=0. 

Essentially, $x_i = \frac{\pi}{2}\frac{i}{N-1}$ for i from 0 to N-1=npoints

So, the expression for G becomes,
\begin{equation}
    G = (-\langle c| A^T + \langle b|)(|b\rangle - A |c\rangle) = \langle b| b \rangle - \langle c| A^T | b \rangle - \langle b|A|c \rangle + \langle c| A^T A| c \rangle  
\end{equation}
Our goal is to find the coefficients $c_n$ that are stored in $|c\rangle$ that minimize $G$, hence we want something like $\frac{\partial G}{\partial |c\rangle}=0$, of course this is a figuratively way of saying that we will numerically divide by $|c\rangle$ and take the outcome equal to zero and solve for $|c\rangle$. Then we will find,
\begin{equation}
    -2 \langle b|A + 2\langle c| A^T A = 0 \rightarrow A^T A |c\rangle = A^T |b\rangle \rightarrow |c\rangle = (A^T A)  \backslash  (A^T |b\rangle)
\end{equation}

Let's see how to compute it numerically.


In [87]:
# function we want to approximate
fstar(x) = sin(x)

fstar (generic function with 1 method)

In [88]:
# the power of polynomial approximation
pmax = 9

9

In [89]:
# evaluation points, here we will use 1000 points, so x[1]=0 and x[1000]=π/2. Julia counts the elements of a vector from 1 not 0.
npoints = 1000
x = LinRange(0, π/2, npoints);

In [90]:
# our matrix $x_i^p$
A = [x[i]^p for i in 1:npoints, p in 0:pmax];

In [91]:
# the vector |b>
b = fstar.(x);

In [92]:
# solving for the vector of coefficients \c>
c = (A'*A) \ (A'*b)

10-element Vector{Float64}:
 -2.1652081292613568e-10
  1.000000011315686
 -1.376446860341138e-7
 -0.16666600266109116
 -1.4564303930903547e-6
  0.008334511007072005
  8.425016946820086e-7
 -0.00020089190594001482
  1.94403055806675e-6
  2.1648773751122687e-6

$p_{max}$ is 9 but we want 10 coefficients since our expansion begins from p=0.

In [93]:
# we define the expansion function
f(c, x) = sum(c[p+1] * x^p for p in 0:pmax)

f (generic function with 1 method)

If we plot together the target function $f^*(x)$ and the expansion with coefficients the ones we found $ g(x) = \sum_{n=0}^{n_{max}} c_n x^n$ for more values of the variable x in $[0,\pi/2]$ we see that for $p_{max}=9$ they coinside. Actually, the series starts converging very fast for even smaller values of $p_{max}$ which one can check by changing it and running the cells again.

In [139]:
fig = Figure()
Axis(fig[1,1],xlabel = "x", ylabel = "f(x)")
xx = LinRange(0, π/2, 2000)
yy = [f(c,x) for x in xx]
lines!(xx, fstar.(xx), color=:brown)
lines!(xx, yy,  color=:blue)
#scatter!(x, [f(c,x′) for x′ in x],  color=:green)
fig

LoadError: UndefVarError: savefig not defined

The error is indeed very small as shown below where we plot the difference $f^*(x)-g(x)$.

LoadError: UndefVarError: savefig not defined

In [95]:
#the error
fig = Figure()
Axis(fig[1,1],xlabel = "x", ylabel = "Error")
xx = LinRange(0, π/2, 2000)
yy = [f(c,x) - fstar(x) for x in xx]
lines!(xx, yy, color=:darkcyan)
fig

##  <span style="color:brown"> 2. Define the approximation error E as the L2 norm of the difference between the approximation function f(x) and fstar(x). This is also called RMS or "root mean square" of the difference: The square root of the average of (f(x) - fstar(x))^2 at all evaluation points. What polynomial order (pmax) is required to satisfy E < 1.0e-10?Plot the relation between pmax and E for pmax from 0 to at least 20. Discuss the figure. Can you achieve E < 1.0e-20? Say why. </span></p>

The L2 Norm is defined as:

E=$\sqrt{\frac{\sum_{i=0, pi/2} ((f^*(xi)-f(c,xi))^2)} {(npoints)}}$

In [96]:
# we define it:
E = sqrt(sum(((f(c,x′)-fstar(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))
# for pmax=10, E= 3.223812199083237e-10
# for pmax=8, E= 5.32618130804448e-10
# for pmax=9, E= 7.121103537354954e-11

7.121103537354954e-11

We try different values of $p_{max}$ but find that for  $p_{max}=9$ we satisfy $E<10^{-10}$. Below we plot the L2 norm error as a function of $p_max$ for $p_{max}$ in 0 to 30.

In [97]:
plim=30
npoints = 1000
x = LinRange(0, π/2, npoints)
Er = zeros(plim +1 )
for pm in 0:plim
    a = [x[i]^p for i in 1:npoints, p in 0:pm]
    d = (a'*a) \ (a'*b)
    fe(d, x) = sum(d[p+1] * x^p for p in 0:pm)
    e = sqrt(sum(((fe(d,x′)-fstar(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))
    Er[pm + 1] = e
end

In [98]:
Er;

In [99]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 0:plim
    scatter!(pm , Er[pm+1], color=:red, linewidth=1)
end
fig

In [100]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 7:plim
    scatter!(pm , Er[pm+1], color=:red, linewidth=1)
end
fig

In [101]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 9:plim
    scatter!(pm , Er[pm+1], color=:red, linewidth=1)
end
fig

We can find for which $p_{max}$ we have the minimum error and which value of the error is this.

In [102]:
# Find the index of the minimum value
min_idx = argmin(Er)

# Print the minimum value and its index
println("Minimum value: ", Er[min_idx])
println("Index of minimum value: ", min_idx)

Minimum value: 7.121103537354954e-11
Index of minimum value: 10


Indeed is the values $p_{max}=9$, remember Er[10]=Er[$p_{max}=9$] (Julia counts from 1). So we cannot achieve $E<10^{-20}$

##  <span style="color:brown"> 3. The function sin(x) is antisymmetric, i.e. sin(x) = - sin(-x). Modify the method to use only antisymmetric polynomials in your approximation. How does this affect the error? Compare results for the same computational cost, i.e. for the same number of polynomials used (not for the same pmax). < 1.0e-20? Say why. </span></p>

Since our target function $f^*(x)$ is antisymmetric it makes sense to try approximate it using only antisymmetric polynomials. We will use the same number of polynomials, not the same $p_{max}$, so the power of polynomial approximation is $2p_{max}+1$.

In [103]:
# function we want to approximate
fstar(x) = sin(x)

fstar (generic function with 1 method)

In [104]:
# the power of polynomial approximation is 2*pmaxo+1 !!! 
pmaxo = 9

9

In [105]:
# evaluation points
npoints = 1000
x = LinRange(0, π/2, npoints);

In [106]:
# matrix
Ao = [x[i]^(2*p+1) for i in 1:npoints, p in 0:pmax];

In [107]:
b=fstar.(x);

In [108]:
co = (Ao'*Ao) \ (Ao'*b);

In [109]:
fo(co, x) = sum(co[p+1] * x^(2*p+1) for p in 0:pmaxo)

fo (generic function with 1 method)

Plotting the approximation against the real values, we see that the curves fall on each other again. The error is very small in this case to and of the same order of magnitude.

In [110]:
fig = Figure()
Axis(fig[1,1],xlabel = "x", ylabel = "f(x)")
xx = LinRange(0, π/2, 2000)
yyo = [fo(co,x) for x in xx]
lines!(xx, fstar.(xx), color=:brown)
lines!(xx, yyo,  color=:blue)
#scatter!(x, [f(c,x′) for x′ in x],  color=:green)
fig

In [111]:
#the error
fig = Figure()
Axis(fig[1,1],xlabel = "x", ylabel = "Error")
xx = LinRange(0, π/2, 2000)
yyo = [fo(co,x) - fstar(x) for x in xx]
lines!(xx, yyo, color=:darkcyan)
fig

In [112]:
Eo = sqrt(sum(((fo(co,x′)-fstar(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))

7.720214533237634e-11

We evaluate the L2 norm again for the same number of polynomials as before, starting from 0 to 30, but we do not find $E<10^{-20}$ in this case either.

In [113]:
plim=30
npoints = 1000
x = LinRange(0, π/2, npoints)
Ero = zeros(plim +1 )
for pm in 0:plim
    ao = [x[i]^(2*p+1) for i in 1:npoints, p in 0:pm]
    d1 = (ao'*ao) \ (ao'*b)
    feo(d1, x) = sum(d1[p+1] * x^(2*p+1) for p in 0:pm)
    eo = sqrt(sum(((feo(d1,x′)-fstar(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))
    Ero[pm + 1] = eo
end

In [114]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 0:plim
    scatter!(pm , Ero[pm+1], color=:violetred4, linewidth=1)
end
fig

In [115]:
Er, Ero;

In [116]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 7:plim
    scatter!(pm , Ero[pm+1], color=:violetred4, linewidth=1)
end
fig

In [117]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 9:plim
    scatter!(pm , Ero[pm+1], color=:violetred4, linewidth=1)
end
fig

In [118]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 9:plim
    scatter!(pm , Ero[pm+1], color=:violetred4, linewidth=1)
    scatter!(pm , Er[pm+1], color=:red, linewidth=1)
end
fig

In [119]:
fig = Figure()
Axis(fig[1,1],xlabel = "pmax", ylabel = "Error")
for pm in 9:plim
    scatter!(pm , Er[pm+1]-Ero[pm+1], color=:slateblue, linewidth=1)
    
end
fig

In [120]:
# Find the index of the minimum value
min_idx = argmin(Ero)

# Print the minimum value and its index
println("Minimum value: ", Ero[min_idx])
println("Index of minimum value: ", min_idx)

Minimum value: 1.347880137494226e-12
Index of minimum value: 7


Now, the $p_{max}$ which corresponds to the minimum error is for $p_{max}= 13$. 

##  <span style="color:brown"> 4.a. Calculate (manually, not numerically) the derivative of the approximating function f(c, x) = \sum_p c_p x^p. Define a new function g(d, x) = \sum_p d_p x^p which depends on a different set of coefficients |d>. Set g(d, x) = f'(c, x), and solve for the coefficients |d> as a function of the coefficients |c>. Since the derivative is a linear operation, the relation between |d> and |c> can be expressed via a linear operator, the derivative matrix D: |d> = D|c>. Calculate D. </span></p>

In [121]:
# our approximate expansion series is:
f(c, x) = sum(c[p+1] * x^p for p in 0:pmax)

f (generic function with 1 method)

The derivative of $ g(x) = \sum_{n=0}^{n_{max}} c_n x^n$ is:

$g'(x)=\sum_{n=1}^{n_{max}} nc_n x^{n-1}=\sum_{n=0}^{n_{max}-1} (n+1)c_{n+1} x^{n}$

In [122]:
# if one took the derivative with respect to x they would find:
df(c,x) = sum((p+1)*c[p+2] * x^(p) for p in 0:(pmax-1))

df (generic function with 1 method)

We define a new function g(d,x) which is a series expansion and we will find the coefficients d s.t. this series approximates our derivative, so we need to find the relation between d and c. It is obvious that $d_p=(p+1)c_{p+1}$ for p from 0 to $p_{max}-1$.

In [123]:
g(d,x) = sum(d[p+1]x^p for p in 0:(pmax-1))

g (generic function with 1 method)

In [124]:
d = zeros(9)
for p in 0:(pmax-1)
    d[p+1]=(p+1)c[p+2]
end
d;

The first element of |c> can be beglected since it does not contribute to the expansion of the derivative, its coefficient is zero. So we work with all the other 9 elements. The same holds true with the matrix A, we can neglect the first column which corresponds to the p=0 of the initial expansion series. We can express the derivative as a diagonal ($p_{max}-1$ \times $p_{max}-1$ ) matrix, with the diagonal term [i,i] being [i+1,i+1] from i from 0 to $p_{max}-1$.

In [125]:
v=c[2:end]
a = A[:, 2:end]
D = zeros(pmax,pmax)
for p in 0:(pmax-1)
    D[p+1,p+1]=(p+1)
end
D

9×9 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  6.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  7.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  8.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0

In [126]:
a = A[:, 2:end];

##  <span style="color:brown"> 4.b. Calculate D numerically, and test your code by comparing (1) approximating sin(x) and calculating the derivative via D, and (2) approximating sin'(x) = cos(x). (Note that cos(x) cannot be approximated well by antisymmetric polynomials, i.e. don't use your code from task 3 above.) </span></p>

In [127]:
# we defined the matrix D above.
D;

Approximating sin'(x)=cos(x), we will use pmax=8 for this approximation to be of the same order as the derivative. We follow the exact same steps as in the first question. 


In [128]:
# the target function
fcos(x) = cos(x)

fcos (generic function with 1 method)

In [129]:
pmaxc=8
npoints = 1000
x = LinRange(0, π/2, npoints);

In [130]:
# matrix
B = [x[i]^p for i in 1:npoints, p in 0:pmaxc];

In [131]:
bc=fcos.(x);

In [132]:
cc = (B'*B) \ (B'*bc)

9-element Vector{Float64}:
  0.999999997988613
  1.1804420669345944e-7
 -0.5000016580060084
  9.78075761600285e-6
  0.04163682064361689
  5.161579802031607e-5
 -0.0014407972499326997
  2.913959802471707e-5
  1.728860100850841e-5

In [133]:
# approximating cos(x) as a polynomial series
fc(cc, x) = sum(cc[p+1] * x^p for p in 0:pmaxc)

fc (generic function with 1 method)

In [144]:
# we plot the approximation polynomial series against the function and see that they coinside as well with grat precision
fig = Figure()
Axis(fig[1,1])
xx = LinRange(0, π/2, 2000)
yyc = [fc(cc,x) for x in xx]
lines!(xx, fcos.(xx), color=:brown)
lines!(xx, yyc,  color=:blue)
#scatter!(x, [f(c,x′) for x′ in x],  color=:green)
fig

Now, let's compare the approximating series for cos(x) with the derivative of the series of sin(x). The error in all cases is very small of the order of $10^{-8} $ to  $10^{-11} $ depending on the $p_{max}$ one chooses. We do not achieve $E<10^{-20}$

In [135]:
fig = Figure()
Axis(fig[1,1])
xx = LinRange(0, π/2, 2000)
yycap = [fc(cc,x) for x in xx]
yycder=[g(d,x) for x in xx]
lines!(xx, yycder, color=:brown)
lines!(xx, yycap,  color=:blue)
#scatter!(x, [f(c,x′) for x′ in x],  color=:green)
fig

In [136]:
Eapp = sqrt(sum(((fc(cc,x′)-fcos(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))

5.331618930140818e-10

In [137]:
Eder = sqrt(sum(((g(d,x′)-fcos(x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))

2.6721023652879887e-9

In [145]:
Eder = sqrt(sum(((g(d,x′)-fc(cc,x′))^2)/(npoints) for x′ in LinRange(0, π/2, npoints)))

2.6043328365386086e-9

However, the approximation found through approximting cos(x) with Least Squared method seems to be more accurate than taking the derivative of sin(x).